In [1]:
import os
import hail as hl
import pyspark
import bokeh
import logging
import random
import pandas as pd
import numpy as np
from scipy import stats
import pickle 
from matplotlib import pyplot as plt
from typing import Any, Counter, List, Optional, Tuple, Union
from hail.plot import show, output_notebook
tmp_dir = "hdfs://spark-master:9820/"
temp_dir = "file:///home/ubuntu/data/tmp"
plot_dir = "/home/ubuntu/data/tmp"

sc = pyspark.SparkContext()
hadoop_config = sc._jsc.hadoopConfiguration()
hadoop_config.set("fs.s3a.access.key", "8YY584J59H7Q6AVKHSU8")
hadoop_config.set("fs.s3a.secret.key", "P8vePa7JUvxKXX2me9ti1cGujgYWMoimAwx4mMlM")
hadoop_config.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_config.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hl.init(sc=sc, tmp_dir=tmp_dir, default_reference='GRCh38')
output_notebook()
logging.basicConfig(format="%(levelname)s (%(name)s %(lineno)s): %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.5
SparkUI available at http://spark-master:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.41-b8144dba46e6
LOGGING: writing to /home/ubuntu/data/tmp/scripts/sanger_gnomad_hail_qc/notebooks/hail-20201012-1600-0.2.41-b8144dba46e6.log


Loading BokehJS ...

In [2]:
ht= hl.read_table(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/aa7a0685/rf_final.ht')

In [3]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        InbreedingCoeff: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
    'rf_hash': str 
    'rf_snv_cutoff': struct {
        min_score: float64
    } 
    'rf_indel_cutoff': struct {
        min_score: float64
    } 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float64 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS

In [6]:
ht.row.show()

locus,alleles,a_index,was_split,InbreedingCoeff,variant_type,allele_type,n_alt_alleles,was_mixed,has_star,AS_QD,AS_MQRankSum,AS_SOR,AS_ReadPosRankSum,hapmap,omni,mills,kgp_phase1_hc,transmitted_singleton,fp,rf_train,rf_label,rf_test,rf_probability,rf_prediction,ac_raw,ac,filters,rf_positive_label,rf_negative_label
locus<GRCh38>,array<str>,int32,bool,float64,str,str,int32,bool,bool,array<float64>,array<float64>,array<float64>,array<float64>,bool,bool,bool,bool,bool,bool,bool,str,bool,float64,str,int32,int32,set<str>,bool,bool
chr1:13024,"[""G"",""A""]",1,false,-1.47e-01,"""snv""","""snv""",1,false,false,[3.10e+01],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,3.16e-01,"""FP""",1,1,"{""AC0"",""RF""}",false,true
chr1:13087,"[""A"",""G""]",1,false,-1.26e-01,"""snv""","""snv""",1,false,false,[7.23e+00],NA,NA,NA,NA,NA,NA,NA,NA,false,NA,NA,false,3.17e-01,"""FP""",1,1,"{""RF""}",false,false
chr1:13176,"[""G"",""T""]",1,false,-1.20e-01,"""snv""","""snv""",1,false,false,[1.12e+01],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,3.17e-01,"""FP""",1,1,"{""AC0"",""RF""}",false,true
chr1:13216,"[""C"",""G""]",1,false,-8.65e-02,"""snv""","""snv""",1,false,false,[1.07e+01],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,3.51e-01,"""FP""",1,1,"{""AC0"",""RF""}",false,true
chr1:13248,"[""C"",""G""]",1,false,-8.23e-02,"""snv""","""snv""",1,false,false,[8.93e+00],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,3.70e-01,"""FP""",1,1,"{""RF""}",false,true
chr1:13255,"[""C"",""T""]",1,false,-8.85e-02,"""snv""","""snv""",1,false,false,[1.65e+00],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,3.51e-01,"""FP""",1,1,"{""RF""}",false,true
chr1:13273,"[""G"",""C""]",1,false,2.23e-01,"""snv""","""snv""",1,false,false,[1.60e+01],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,6.49e-01,"""TP""",1,1,"{""RF""}",false,true
chr1:13279,"[""C"",""A""]",1,false,-6.36e-02,"""snv""","""snv""",1,false,false,[3.15e+00],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,3.76e-01,"""FP""",1,1,"{""AC0"",""RF""}",false,true
chr1:13281,"[""C"",""G""]",1,false,-5.97e-02,"""snv""","""snv""",1,false,false,[4.50e+00],NA,NA,NA,NA,NA,NA,NA,NA,true,true,"""FP""",false,3.76e-01,"""FP""",1,1,"{""AC0"",""RF""}",false,true


In [6]:
ht.count()

929613

In [8]:
ht.globals.test_results.show()

test_results
"array<struct{rf_prediction: str, rf_label: str, n: int32}>"
"[(""FP"",""FP"",4501),(""FP"",""TP"",21389),(""TP"",""FP"",2264),(""TP"",""TP"",42193)]"


In [2]:
ht = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED_FREQ_adj.ht')

In [3]:
ht.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
     

In [4]:
ht=ht.annotate(InbreedingCoeff=ht.info.AS_InbreedingCoeff)

In [5]:
ht.write(f'{tmp_dir}/Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED_FREQ_adj_inb.ht', overwrite=True)

2020-10-12 15:42:00 Hail: INFO: wrote table with 4640042 rows in 21200 partitions to hdfs://spark-master:9820//Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED_FREQ_adj_inb.ht


In [2]:
run_hash="aa7a0685"
rf_model = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/training.ht')

In [11]:
ht = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED_FREQ_adj_inb.ht')
        #ht = get_rf(data="training", run_hash=run_hash).ht()
features = hl.eval(rf_model.features)

In [12]:
ht.count()

4640042

In [6]:
features

['InbreedingCoeff',
 'variant_type',
 'allele_type',
 'n_alt_alleles',
 'was_mixed',
 'has_star']

In [7]:
ht.describe()

----------------------------------------
Global fields:
    'freq_meta': array<dict<str, str>> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        FS: float64, 
        InbreedingCoeff: float64, 
        MLEAC: array<int32>, 
        MLEAF: array<float64>, 
        MQ: float64, 
        MQRankSum: float64, 
        QD: float64, 
     

In [8]:
ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_table_for_RF_by_variant_type.ht')

In [9]:
ht.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        InbreedingCoeff: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float64 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
    'transmitted_singleton': bool 
    'fail_hard_filters': bool 
    'feature_imputed': struct {
        a_index: bool, 
        InbreedingCoeff: bool, 
        n_alt_alleles: bool
    } 
-

In [10]:
ht.count()

929613

In [14]:
ht = hl.read_table(
         f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_table_for_RF.ht')

In [15]:
ht.count()

929613

In [16]:
# ANNOTATION TABLES:
truth_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/truthset_table.ht')
trio_stats_table = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_trios_stats.ht')
    #inbreeding_ht = hl.read_table(f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_inbreeding.ht')
allele_data_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_allele_data.ht')
allele_counts_ht = hl.read_table(
        f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_qc_ac.ht')


In [17]:
group = "raw"
mt = hl.read_matrix_table(
        f'{temp_dir}/ddd-elgh-ukbb/filtering/Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED.mt')
mt = mt.select_entries(
        GT=hl.unphased_diploid_gt_index_call(mt.GT.n_alt_alleles()))
mt = mt.annotate_rows(InbreedingCoeff=hl.or_missing(
        ~hl.is_nan(mt.info.InbreedingCoeff), mt.info.InbreedingCoeff))
ht = mt.rows()
ht = ht.transmute(**ht.info)
ht = ht.select("FS", "MQ", "QD", "InbreedingCoeff", *INFO_FEATURES)

NameError: name 'INFO_FEATURES' is not defined

In [18]:
INFO_FEATURES = [
    "AS_QD",
    "AS_ReadPosRankSum",
    "AS_MQRankSum",
    "AS_SOR",
]  # Note: AS_SOR is currently in VQSR HT and named SOR in the VQSR split HT

In [19]:
ht = ht.select("FS", "MQ", "QD", "InbreedingCoeff", *INFO_FEATURES)

In [20]:
ht.count()

4640042

In [21]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'FS': float64 
    'MQ': float64 
    'QD': float64 
    'InbreedingCoeff': float64 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [22]:
trio_stats_ht = trio_stats_table.select(
        f"n_transmitted_{group}", f"ac_children_{group}"
    )

ht = ht.annotate(
        **trio_stats_ht[ht.key],
        **truth_data_ht[ht.key],
        **allele_data_ht[ht.key].allele_data,
        **allele_counts_ht[ht.key],
    )

In [23]:
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'FS': float64 
    'MQ': float64 
    'QD': float64 
    'InbreedingCoeff': float64 
    'AS_QD': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'n_transmitted_raw': int64 
    'ac_children_raw': int64 
    'hapmap': bool 
    'omni': bool 
    'kgp_phase1_hc': bool 
    'mills': bool 
    'nonsplit_alleles': array<str> 
    'has_star': bool 
    'variant_type': str 
    'n_alt_alleles': int32 
    'allele_type': str 
    'was_mixed': bool 
    'rsid': str 
    'qual': float64 
    'filters': set<str> 
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: arra

In [24]:
 ht = ht.checkpoint(
        f'{tmp_dir}/ddd-elgh-ukbb/Sanger_cohorts_for_RF_unfiltered.ht', overwrite=True)

2020-10-12 16:15:29 Hail: INFO: wrote table with 4640042 rows in 21200 partitions to hdfs://spark-master:9820//ddd-elgh-ukbb/Sanger_cohorts_for_RF_unfiltered.ht


In [25]:
rf_model = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/models/{run_hash}/training.ht')
        # ht = hl.read_table(
        #    f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_chr1-20-XY_sampleQC_FILTERED_FREQ_adj_inb.ht')
ht = hl.read_table(
            f'{temp_dir}/ddd-elgh-ukbb/variant_qc/Sanger_cohorts_for_RF_unfiltered.ht')

In [26]:
rf_model.describe()

----------------------------------------
Global fields:
    'feature_medians': dict<tuple (
        str
    ), struct {
        a_index: int32, 
        InbreedingCoeff: float64, 
        n_alt_alleles: int32
    }> 
    'variants_by_strata': dict<tuple (
        str
    ), int64> 
    'features_importance': dict<str, float64> 
    'features': array<str> 
    'test_results': array<struct {
        rf_prediction: str, 
        rf_label: str, 
        n: int32
    }> 
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'a_index': int32 
    'was_split': bool 
    'InbreedingCoeff': float64 
    'variant_type': str 
    'allele_type': str 
    'n_alt_alleles': int32 
    'was_mixed': bool 
    'has_star': bool 
    'AS_QD': array<float64> 
    'AS_MQRankSum': array<float64> 
    'AS_SOR': array<float64> 
    'AS_ReadPosRankSum': array<float64> 
    'hapmap': bool 
    'omni': bool 
    'mills': bool 
    'kgp_phase1_hc': bool 
  

In [27]:
ht=ht.annotate(rf_label=rf_model[ht.key].rf_label)